In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .master('local[8]')
    .config('spark.driver.memory', '16g')
    .getOrCreate()
)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/17 00:46:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
test_data = spark.read.parquet('../data/amex-default-prediction/test_data')
train_data = spark.read.parquet('../data/amex-default-prediction/train_data')
train_labels = spark.read.parquet('../data/amex-default-prediction/train_labels')

In [3]:
import pandas as pd
import warnings 

pd.options.display.max_columns = None
pd.options.display.max_rows = None
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [4]:
test_data.describe().toPandas().set_index('summary').transpose()

22/06/17 00:50:21 WARN DAGScheduler: Broadcasting large task binary with size 1248.5 KiB
22/06/17 00:50:23 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


summary,count,mean,stddev,min,max
customer_ID,11363762,None,None,00000469ba478561f23a92a868bd366de6f6527a684c9a...,fffffa7cf7e453e1acc6a1426475d5cb9400859f82ff61...
P_2,11303884,0.6576888706945034,0.24689157666773695,-0.46585518,1.01
D_39,11363762,0.15627541796240602,0.2657579252159225,6.20427E-9,9.330448
B_1,11363762,0.12794969068492926,0.21268946844905062,-7.0574164,1.3240603
B_2,11360845,0.6167632378054201,0.4059171866636613,2.1250483E-10,1.01
R_1,11363762,0.07489555064771232,0.2160619377089204,6.9110545E-10,3.2585073
S_3,9622797,0.2274024128843623,0.1988264517135985,-0.685562,4.341215
D_41,11360845,0.05915284930308632,0.20069504741751648,4.726168E-10,12.117185
B_3,11360845,0.1433975302600992,0.2451063383832772,4.880305E-10,1.6986513
D_42,1357583,0.14808572005712342,0.2511165586847569,-5.46781E-4,4.1911583


In [5]:
train_data.describe().toPandas().set_index('summary').transpose()

22/06/17 00:52:07 WARN DAGScheduler: Broadcasting large task binary with size 1249.8 KiB


summary,count,mean,stddev,min,max
customer_ID,5531451,None,None,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...
P_2,5485466,0.6563339688868546,0.24464941706021068,-0.45895484,1.01
D_39,5531451,0.1531171838713319,0.27007087581992834,5.02619E-9,5.3896194
B_1,5531451,0.12401000613035297,0.21198689209071642,-7.588799,1.3240604
B_2,5529435,0.6214887020829851,0.4014876516204405,9.19228E-9,1.01
R_1,5531451,0.07880269570337799,0.22639713083808122,1.534223E-9,3.2562845
S_3,4510907,0.22584545103006262,0.19334747119451923,-0.62713206,5.482888
D_41,5529435,0.05978469262130524,0.202544251029949,5.566545E-10,8.988807
B_3,5529435,0.13253894272418434,0.23499286904456476,6.285293E-9,1.6252623
D_42,791314,0.18497419533974818,0.22818457987906945,-4.5433026E-4,4.1911187


In [6]:
train_labels.describe().toPandas().set_index('summary').transpose()

summary,count,mean,stddev,min,max
customer_ID,458913,None,None,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...
target,458913,0.2589336105100531,0.43804955654914784,0.0,1.0
